In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the datasets from web scraper
df1 = pd.read_csv('IndeedDataAnalyst_2023.csv')
df2 = pd.read_csv('IndeedDataEngineer_2023.csv')
df3 = pd.read_csv('IndeedDataScientist_2023.csv')
df4 = pd.read_csv('simplyhiredjobs.csv')
df5 = pd.read_csv('dice_data_with_skills.csv')
df_new = pd.read_csv('FinalJobDataUpdated.csv')

In [3]:
# Indeed datasets
df1.insert(0, 'job_query', 'Data Analyst')
df2.insert(0, 'job_query', 'Data Engineer')
df3.insert(0, 'job_query', 'Data Scientist')

In [4]:
# combine datasets together
df = pd.concat([df1, df2, df3])

In [5]:
# format column headers
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()
df = df.drop(columns='unnamed:_0')

In [6]:
#df.head()

In [7]:
# insert new column
df.insert(0, 'job_website_source', 'Indeed')

In [8]:
df.head()

,job_website_source,job_query,job_title,company,location,salary,job_description
0,Indeed,Data Analyst,Board Certified Behavior Analyst (BCBA),Aspire Learning Center,"100 Cummings Center, Beverly, MA 01915",$40 - $70 an hour,"ASPIRE LEARNING CENTER, LLC\nJob Title: Board ..."
1,Indeed,Data Analyst,"Business Analyst II, Research",KeyLogic Systems,"Morgantown, WV",N,KeyLogic Systems has an immediate need for a B...
2,Indeed,Data Analyst,"Business Analyst (New Holland, PA or Napoleon,...",Goodville Mutual Casualty Company,"625 W Main St, New Holland, PA 17557",N,Nature of Job\nServe as subject matter expert ...
3,Indeed,Data Analyst,Data Analyst I,Salus Group,"Sterling Heights, MI 48310•Hybrid remote","$60,829 - $67,720 a year",Salus Group has been around conducting busines...
4,Indeed,Data Analyst,Data Standard Analyst - Biostatistics Team,"Medpace, Inc.","Cincinnati, OH 45227•Hybrid remote",N,Job Summary:\nOur corporate activities are gro...


In [9]:
# Drop rows where 'Salary' is less than 40,000
df = df[df['salary'].str.replace(',', '').str.extract(r'(\d+)', expand=False).fillna('0').astype(int) >= 40000]

# Split 'Salary' column into 'Yearly Min' and 'Yearly Max' columns
df[['Yearly Min', 'Yearly Max']] = df['salary'].str.replace(',', '').str.extract(r'(\d+)\D*(\d+)?', expand=True).astype(float).fillna(0)

# Set 'Yearly Max' to 'Yearly Min' if it is 0
df['Yearly Max'] = df.apply(lambda row: row['Yearly Min'] if row['Yearly Max'] == 0 else row['Yearly Max'], axis=1)

# Remove the 'Salary' column
df = df.drop('salary', axis=1)

# Format 'Yearly Min' and 'Yearly Max' columns as strings with two decimal places
df[['Yearly Min', 'Yearly Max']] = df[['Yearly Min', 'Yearly Max']].applymap('{:.2f}'.format)

In [10]:
# Add a 'Skills' column if one doesn't exist
if 'Skills' not in df.columns:
    df['Skills'] = ''

# Define the list of keywords
keywords = ['SQL', 'Python', 'Big Data', 'AWS', 'ETL', 'Hadoop', 'Spark', 'Kafka', 'Data Warehousing', 'Data Pipelines', 
            'Data Modeling', 'Java', 'Database Management', 'NoSQL', 'Airflow', 'Docker', 'Kubernetes', 'Redshift', 
            'Snowflake', 'Data Integration', 'Excel', 'Tableau', 'Data Visualization', 'Data Analysis', 'Dashboards', 
            'Reporting', 'Business Intelligence', 'Data Mining', 'Statistics', 'Power BI', 'Data Cleansing', 
            'Data Interpretation', 'Google Analytics', 'Data Modelling', 'Predictive Analytics', 'R$', 'Data Mapping', 
            'Machine Learning', 'Deep Learning', 'Natural Language Processing', 'Predictive Modeling', 
            'Mathematical Modeling', 'TensorFlow', 'Keras', 'Computer Vision', 'Artificial Intelligence']

# Loop over the rows in the DataFrame and update the 'Skills' column
for index, row in df.iterrows():
    job_description = row['job_description']
    skills = []
    for keyword in keywords:
        if keyword.lower() in job_description.lower():
            if keyword.lower() == 'r$':
                if 'ruby' in job_description.lower():
                    continue
            skills.append(keyword)
    row['Skills'] = ', '.join(skills)

In [11]:
df.head()

,job_website_source,job_query,job_title,company,location,job_description,Yearly Min,Yearly Max,Skills
3,Indeed,Data Analyst,Data Analyst I,Salus Group,"Sterling Heights, MI 48310•Hybrid remote",Salus Group has been around conducting busines...,60829.00,67720.00,"Excel, Reporting"
5,Indeed,Data Analyst,Technical Data Analyst,"Althea US, Inc.","Goodlettsville, TN 37072•Hybrid remote","As technical data analyst, you will be respons...",65000.00,75000.00,"Excel, Dashboards, Reporting"
7,Indeed,Data Analyst,Data Analyst,"Atmosfy, Inc",Remote,A BIT ABOUT YOU:\nYou're passionate and extrem...,60000.00,90000.00,"SQL, AWS, Excel, Dashboards"
9,Indeed,Data Analyst,Data Analyst - Remote,HSS Inc.,United States,Overview:\nThis position performs data analysi...,70000.00,80000.00,"SQL, Excel, Tableau, Data Analysis, Dashboards..."
12,Indeed,Data Analyst,Entry Level Business Analyst,Microflexinfra,Remote,Microflexinfra focuses on providing business p...,65000.00,85000.00,


In [12]:
# regex pattern to parse city and state from location column
city_pattern = r'([A-Za-z]+(?: [A-Za-z]+)*)'
state_pattern = r'([A-Z]{2})'

In [13]:
# create new columns for city and state
df['city'] = df['location'].str.extract(city_pattern, expand=False)
df['state'] = df['location'].str.extract(state_pattern, expand=False)

In [14]:
df.head(10)

,job_website_source,job_query,job_title,company,location,job_description,Yearly Min,Yearly Max,Skills,city,state
3,Indeed,Data Analyst,Data Analyst I,Salus Group,"Sterling Heights, MI 48310•Hybrid remote",Salus Group has been around conducting busines...,60829.00,67720.00,"Excel, Reporting",Sterling Heights,MI
5,Indeed,Data Analyst,Technical Data Analyst,"Althea US, Inc.","Goodlettsville, TN 37072•Hybrid remote","As technical data analyst, you will be respons...",65000.00,75000.00,"Excel, Dashboards, Reporting",Goodlettsville,TN
7,Indeed,Data Analyst,Data Analyst,"Atmosfy, Inc",Remote,A BIT ABOUT YOU:\nYou're passionate and extrem...,60000.00,90000.00,"SQL, AWS, Excel, Dashboards",Remote,NaN
9,Indeed,Data Analyst,Data Analyst - Remote,HSS Inc.,United States,Overview:\nThis position performs data analysi...,70000.00,80000.00,"SQL, Excel, Tableau, Data Analysis, Dashboards...",United States,NaN
12,Indeed,Data Analyst,Entry Level Business Analyst,Microflexinfra,Remote,Microflexinfra focuses on providing business p...,65000.00,85000.00,,Remote,NaN
15,Indeed,Data Analyst,HR Business Analyst,Premier Staffing Solution,"Washington, DC",Position: HR Technology Business Analyst\nLoca...,80000.00,115000.00,"AWS, Excel, Reporting",Washington,DC
16,Indeed,Data Analyst,Data Analyst,Honor,Remote,Data Analyst\nWe’re looking for Data Analyst t...,92000.00,113000.00,"SQL, Python, Excel, Data Analysis, Reporting",Remote,NaN
18,Indeed,Data Analyst,Business Analyst,KesarWeb,"Detroit, MI•Remote",KesarWeb is a custom software development comp...,84000.00,96000.00,"SQL, Data Visualization, Reporting",Detroit,MI
21,Indeed,Data Analyst,Data Analyst,Wilson Elser,"White Plains, NY","4/3/23 | White Plains, NY\nWilson Elser’s Whit...",65000.00,85000.00,"SQL, Excel, Reporting",White Plains,NY
22,Indeed,Data Analyst,Data Analyst,FanDuel,"New York, NY•Hybrid remote",ABOUT FANDUEL GROUP\nFanDuel Group is a world-...,90000.00,115000.00,"SQL, Python, Excel, Data Visualization, Reporting",New York,NY


In [15]:
# regex pattern for searching hybrid and remote work in location column
hybrid_pattern = r'\bhybrid\b'  # Matches the word "hybrid" surrounded by word boundaries
remote_pattern = r'\bremote\b'  # Matches the word "remote" surrounded by word boundaries

In [16]:
# create new columns with Boolean values
df['hybrid'] = df['location'].str.contains(hybrid_pattern, case=False)
df['remote'] = df['location'].str.contains(remote_pattern, case=False)

In [17]:
df.head(10)

,job_website_source,job_query,job_title,company,location,job_description,Yearly Min,Yearly Max,Skills,city,state,hybrid,remote
3,Indeed,Data Analyst,Data Analyst I,Salus Group,"Sterling Heights, MI 48310•Hybrid remote",Salus Group has been around conducting busines...,60829.00,67720.00,"Excel, Reporting",Sterling Heights,MI,True,True
5,Indeed,Data Analyst,Technical Data Analyst,"Althea US, Inc.","Goodlettsville, TN 37072•Hybrid remote","As technical data analyst, you will be respons...",65000.00,75000.00,"Excel, Dashboards, Reporting",Goodlettsville,TN,True,True
7,Indeed,Data Analyst,Data Analyst,"Atmosfy, Inc",Remote,A BIT ABOUT YOU:\nYou're passionate and extrem...,60000.00,90000.00,"SQL, AWS, Excel, Dashboards",Remote,NaN,False,True
9,Indeed,Data Analyst,Data Analyst - Remote,HSS Inc.,United States,Overview:\nThis position performs data analysi...,70000.00,80000.00,"SQL, Excel, Tableau, Data Analysis, Dashboards...",United States,NaN,False,False
12,Indeed,Data Analyst,Entry Level Business Analyst,Microflexinfra,Remote,Microflexinfra focuses on providing business p...,65000.00,85000.00,,Remote,NaN,False,True
15,Indeed,Data Analyst,HR Business Analyst,Premier Staffing Solution,"Washington, DC",Position: HR Technology Business Analyst\nLoca...,80000.00,115000.00,"AWS, Excel, Reporting",Washington,DC,False,False
16,Indeed,Data Analyst,Data Analyst,Honor,Remote,Data Analyst\nWe’re looking for Data Analyst t...,92000.00,113000.00,"SQL, Python, Excel, Data Analysis, Reporting",Remote,NaN,False,True
18,Indeed,Data Analyst,Business Analyst,KesarWeb,"Detroit, MI•Remote",KesarWeb is a custom software development comp...,84000.00,96000.00,"SQL, Data Visualization, Reporting",Detroit,MI,False,True
21,Indeed,Data Analyst,Data Analyst,Wilson Elser,"White Plains, NY","4/3/23 | White Plains, NY\nWilson Elser’s Whit...",65000.00,85000.00,"SQL, Excel, Reporting",White Plains,NY,False,False
22,Indeed,Data Analyst,Data Analyst,FanDuel,"New York, NY•Hybrid remote",ABOUT FANDUEL GROUP\nFanDuel Group is a world-...,90000.00,115000.00,"SQL, Python, Excel, Data Visualization, Reporting",New York,NY,True,True


In [18]:
# rename columns for final dataset
df = df.rename(columns={"job_website_source": "Job Website",
                   "job_query": "Search Parameter",
                   "job_title": "Job Title",
                   "company": "Company",
                  "location": "Location",
                  "job_description": "Job Description",
                  "city": "City",
                   "state": "State Code",
                   "hybrid": "Hybrid",
                   "remote": "Remote",
                  })

In [19]:
df.head()

,Job Website,Search Parameter,Job Title,Company,Location,Job Description,Yearly Min,Yearly Max,Skills,City,State Code,Hybrid,Remote
3,Indeed,Data Analyst,Data Analyst I,Salus Group,"Sterling Heights, MI 48310•Hybrid remote",Salus Group has been around conducting busines...,60829.00,67720.00,"Excel, Reporting",Sterling Heights,MI,True,True
5,Indeed,Data Analyst,Technical Data Analyst,"Althea US, Inc.","Goodlettsville, TN 37072•Hybrid remote","As technical data analyst, you will be respons...",65000.00,75000.00,"Excel, Dashboards, Reporting",Goodlettsville,TN,True,True
7,Indeed,Data Analyst,Data Analyst,"Atmosfy, Inc",Remote,A BIT ABOUT YOU:\nYou're passionate and extrem...,60000.00,90000.00,"SQL, AWS, Excel, Dashboards",Remote,NaN,False,True
9,Indeed,Data Analyst,Data Analyst - Remote,HSS Inc.,United States,Overview:\nThis position performs data analysi...,70000.00,80000.00,"SQL, Excel, Tableau, Data Analysis, Dashboards...",United States,NaN,False,False
12,Indeed,Data Analyst,Entry Level Business Analyst,Microflexinfra,Remote,Microflexinfra focuses on providing business p...,65000.00,85000.00,,Remote,NaN,False,True


In [20]:
df.dtypes

Job Website         object
Search Parameter    object
Job Title           object
Company             object
Location            object
Job Description     object
Yearly Min          object
Yearly Max          object
Skills              object
City                object
State Code          object
Hybrid                bool
Remote                bool
dtype: object

In [21]:
# convert salary columns from string to float
df['Yearly Min'] = df['Yearly Min'].astype(float)
df['Yearly Max'] = df['Yearly Max'].astype(float)

In [22]:
#df.head()

In [23]:
df.dtypes

Job Website          object
Search Parameter     object
Job Title            object
Company              object
Location             object
Job Description      object
Yearly Min          float64
Yearly Max          float64
Skills               object
City                 object
State Code           object
Hybrid                 bool
Remote                 bool
dtype: object

In [24]:
# start of df4 SimplyHired data cleaning

In [25]:
df4.insert(0, 'job_query', 'Data Engineer')
df4.insert(0, 'job_website_source', 'SimplyHired')

In [26]:
# Convert the salary column to numeric values, replace non-numeric values with NaN
df4['sal_min'] = pd.to_numeric(df4['sal_min'], errors='coerce')

# Remove any rows where salary is NaN
df4 = df4.dropna(subset=['sal_min'])

In [27]:
# Convert the salary column to numeric values, replace non-numeric values with NaN
df4['sal_max'] = pd.to_numeric(df4['sal_max'], errors='coerce')

# Remove any rows where salary is NaN
df4 = df4.dropna(subset=['sal_max'])

In [28]:
# convert hourly rates to yearly salary

def hourly_to_yearly(hourly_rate):
    return hourly_rate * 40 * 52  # 40 hours per week, 52 weeks per year

# Create a new 'yearly_salary' column based on the values in 'salary_type' and 'salary'
df4['Yearly Min'] = df4.apply(lambda x: x['sal_min'] if x['salary_type'] == 'yearly' else hourly_to_yearly(x['sal_min']), axis=1)
df4['Yearly Max'] = df4.apply(lambda x: x['sal_max'] if x['salary_type'] == 'yearly' else hourly_to_yearly(x['sal_max']), axis=1)

In [29]:
df4 = df4.drop('index', axis=1)

In [30]:
# regex pattern to search for hybrid and remote work
hybrid_pattern = r'\bhybrid\b'  # Matches the word "hybrid" surrounded by word boundaries
remote_pattern = r'\bremote\b'  # Matches the word "remote" surrounded by word boundaries

In [31]:
df4['hybrid'] = df4['type'].str.contains(hybrid_pattern, case=False)
df4['remote'] = df4['type'].str.contains(remote_pattern, case=False)

In [32]:
df4 = df4.rename(columns={"job_website_source": "Job Website",
                   "job_query": "Search Parameter",
                   "title": "Job Title",
                   "company": "Company",
                  "location": "Location",
                    "skills": "Skills",
                  "job_description": "Job Description",
                  "city": "City",
                   "state": "State Code",
                   "hybrid": "Hybrid",
                   "remote": "Remote",
                  })

In [33]:
df4 = df4.drop(['type', 'salary', 'salary_type', 'sal_min', 'sal_max', 'sal_median', 'posted_date', 'origin'], axis=1)

In [34]:
# combine Indeed dataset with SimplyHired
df = pd.concat([df, df4])

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1715 entries, 3 to 833
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Job Website       1715 non-null   object 
 1   Search Parameter  1715 non-null   object 
 2   Job Title         1715 non-null   object 
 3   Company           1715 non-null   object 
 4   Location          1051 non-null   object 
 5   Job Description   1051 non-null   object 
 6   Yearly Min        1715 non-null   float64
 7   Yearly Max        1715 non-null   float64
 8   Skills            1149 non-null   object 
 9   City              1534 non-null   object 
 10  State Code        1256 non-null   object 
 11  Hybrid            1715 non-null   bool   
 12  Remote            1715 non-null   bool   
dtypes: bool(2), float64(2), object(9)
memory usage: 164.1+ KB


In [36]:
#df.tail()

In [37]:
# start of df5 Dice data cleaning

In [38]:
df5.insert(0, 'job_query', 'Data Scientist')
df5.insert(0, 'job_website_source', 'dice.com')

In [39]:
def hourly_to_yearly(hourly_rate):
    return hourly_rate * 40 * 52  # 40 hours per week, 52 weeks per year

# Create a new 'yearly_salary' column based on the values in 'salary_type' and 'salary'
df5['Yearly Min'] = df5.apply(lambda x: x['sal_min'] if x['type.1'] == 'yearly' else hourly_to_yearly(x['sal_min']), axis=1)
df5['Yearly Max'] = df5.apply(lambda x: x['sal_max'] if x['type.1'] == 'yearly' else hourly_to_yearly(x['sal_max']), axis=1)

In [40]:
hybrid_pattern = r'\bhybrid\b'  # Matches the word "hybrid" surrounded by word boundaries
remote_pattern = r'\bremote\b'  # Matches the word "remote" surrounded by word boundaries

In [41]:
df5['hybrid'] = df5['title'].str.contains(hybrid_pattern, case=False)
df5['remote'] = df5['title'].str.contains(remote_pattern, case=False)

In [42]:
df5 = df5.rename(columns={"job_website_source": "Job Website",
                   "job_query": "Search Parameter",
                   "title": "Job Title",
                   "company": "Company",
                  "location": "Location",
                  "job_description": "Job Description",
                  "city": "City",
                   "state": "State Code",
                   "hybrid": "Hybrid",
                   "remote": "Remote",
                  })

In [43]:
df5 = df5.drop(['index', 'posted_date', 'type', 'salary', 'type.1', 'sal_min', 'sal_max', 'sal_median', 'origin'], axis=1)

In [44]:
df5 = df5.dropna(subset=['Yearly Min'])
df5 = df5.dropna(subset=['Yearly Max'])

In [45]:
#df5

In [46]:
# create final dataset with Indeed, SimplyHired, and Dice all together

df = pd.concat([df, df5])

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1972 entries, 3 to 795
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Job Website       1972 non-null   object 
 1   Search Parameter  1972 non-null   object 
 2   Job Title         1972 non-null   object 
 3   Company           1972 non-null   object 
 4   Location          1051 non-null   object 
 5   Job Description   1051 non-null   object 
 6   Yearly Min        1972 non-null   float64
 7   Yearly Max        1972 non-null   float64
 8   Skills            1361 non-null   object 
 9   City              1742 non-null   object 
 10  State Code        1465 non-null   object 
 11  Hybrid            1972 non-null   object 
 12  Remote            1972 non-null   object 
dtypes: float64(2), object(11)
memory usage: 215.7+ KB


In [53]:
df.tail()

,Job Website,Search Parameter,Job Title,Company,Location,Job Description,Yearly Min,Yearly Max,Skills,City,State Code,Hybrid,Remote
772,dice.com,Data Scientist,Data Scientist,Robert Half,NaN,NaN,128440.0,148720.0,"It, Engineering, Computer, SQL, Python, Analys...",Houston,TX,False,False
775,dice.com,Data Scientist,Jr. Data Scientist,Skilltune,NaN,NaN,40000.0,55000.0,"emass, xacta",New York,NY,False,False
783,dice.com,Data Scientist,Data Scientist / Data Analyst,"firstPRO, Inc",NaN,NaN,80000.0,100000.0,R,Camden,NJ,False,False
786,dice.com,Data Scientist,Data Scientist (Hybrid Onsite 2 days a week),Xoriant Corporation,NaN,NaN,70000.0,80000.0,"Python, Unix, Oracle",Tampa,FL,True,False
795,dice.com,Data Scientist,Jr. Machine Learning Data Scientist,"K2 Partnering Solutions, Inc.",NaN,NaN,50000.0,60000.0,"Python, ML",NaN,NaN,False,False


In [48]:
df.to_csv('Final_Jobs_Dataset.csv')

In [49]:
# Count the total number of rows in the dataframe -- Each row represents 1 job
num_rows = len(df_new)

print("Total Jobs Analyzed:", num_rows)

#Query, average salary of each search parameter "Data Analyst", "Data Engineer", "Data Scientist"


Total Jobs Analyzed: 1225


In [50]:
df_new['Yearly Max'] = df_new['Yearly Max'].replace(0, 'Null')
# Replace 'Null' values with np.nan
df_new = df_new.replace('Null', np.nan)

# Convert the 'Yearly Max' and 'Yearly Min' columns to float
df_new['Yearly Max'] = df_new['Yearly Max'].astype(float)
df_new['Yearly Min'] = df_new['Yearly Min'].astype(float)

# Calculate the average salary
df_new['Average Salary'] = (df_new['Yearly Max'] + df_new['Yearly Min']) / 2

In [51]:
# Group by each unique value in the 'Search Parameter' column and take the group's average of the "Average Salary" column
grouped_df = df_new.groupby(['Search Parameter'])['Average Salary'].mean()
grouped_df = grouped_df.apply(lambda x: '${:,.2f}'.format(x))

print(grouped_df)

Search Parameter
Data Analyst       $95,744.49
Data Engineer     $136,095.57
Data Scientist    $138,970.21
Name: Average Salary, dtype: object


In [52]:
top_paying_cities = df_new.groupby(['City', 'State Code'])['Average Salary'].mean().sort_values(ascending = False)

print(top_paying_cities)

City                State Code
East Hanover        NJ            265000.0
Foster City         CA            221500.0
Falls Church        VA            210000.0
Andover             MA            205400.0
Annapolis Junction  MD            195000.0
                                    ...   
Boca Raton          FL             50000.0
Taylor              MI             44720.0
Cherry Hill         NJ                 NaN
Spokane             WA                 NaN
Tallahassee         FL                 NaN
Name: Average Salary, Length: 227, dtype: float64
